In [1]:
import sys
import requests
import tarfile
import json
import numpy as np
from os import path
from PIL import Image
from PIL import ImageFont, ImageDraw
from glob import glob
from matplotlib import pyplot as plt
%matplotlib inline
from os import path

In [2]:
import json
import os
import shutil

# Load the annotations JSON file
annotations_path = '/kaggle/input/documnet-layout-recognition-dataset-publaynet-t0/labels/publaynet/train.json'
with open(annotations_path, 'r') as f:
    annotations = json.load(f)

# Extract images and their annotations
image_data = {}
for image_info in annotations['images']:
    image_id = image_info['id']
    file_name = image_info['file_name']
    image_data[image_id] = {
        'file_name': file_name,
        'annotations': []
    }

for ann in annotations['annotations']:
    image_id = ann['image_id']
    image_data[image_id]['annotations'].append(ann)

# Now filter annotations for only the images that are in the dataset
image_dir = '/kaggle/input/documnet-layout-recognition-dataset-publaynet-t0/train-0/publaynet/train'
image_files = {f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))}

filtered_annotations = {image_id: data for image_id, data in image_data.items() if data['file_name'] in image_files}

# Take the first 2000 images
filtered_annotations_8000 = dict(list(filtered_annotations.items())[:8000])

# Create new folder for saving the images and annotations
new_image_dir = '/kaggle/working/train_8000_images'
os.makedirs(new_image_dir, exist_ok=True)

# Copy the images to the new folder
for image_id, data in filtered_annotations_8000.items():
    src_path = os.path.join(image_dir, data['file_name'])
    dst_path = os.path.join(new_image_dir, data['file_name'])
    shutil.copyfile(src_path, dst_path)

# Create a new annotations file in COCO format for the first 2000 images
new_annotations = {
    "images": [],
    "annotations": [],
    "categories": annotations["categories"]  # Assuming categories stay the same
}

# Add the relevant images and annotations to the new annotations file
for image_id, data in filtered_annotations_8000.items():
    # Add image info
    for img_info in annotations['images']:
        if img_info['id'] == image_id:
            new_annotations["images"].append(img_info)
    
    # Add the annotations for this image
    for ann in data['annotations']:
        new_annotations["annotations"].append(ann)

# Save the new annotations file
new_annotations_path = '/kaggle/working/train_8000_annotations.json'
with open(new_annotations_path, 'w') as f:
    json.dump(new_annotations, f)

print(f"Done! Copied 8000 images and saved the annotations to {new_image_dir} and {new_annotations_path}.")


Done! Copied 8000 images and saved the annotations to /kaggle/working/train_8000_images and /kaggle/working/train_8000_annotations.json.


In [3]:
import shutil
import os

# Define the folder path to be zipped
folder_to_zip = '/kaggle/working/train_8000_images'

# Define the output zip file path (without the .zip extension)
output_zip = '/kaggle/working/train_8000_images'

# Create a zip file
shutil.make_archive(output_zip, 'zip', folder_to_zip)

print(f"Zipped folder saved as {output_zip}.zip")


Zipped folder saved as /kaggle/working/train_8000_images.zip
